# LLT Code

### Global Variables
I compute LLT polynomials with a fixed number of variables. If you want infinitely many variables, you should set **nvar** to some big number (the sum of the lengths of all the shapes should suffice). This might slow things down, I haven't tested it...

In [22]:
from itertools import product

F = QQ['t']
K = FractionField(F)
Sym = SymmetricFunctions(K)
nvar = 4 # number of variables
x_var_symbols = var(*['x{0}'.format(i) for i in range(1, nvar+1)])
if nvar == 1:
    x_var_symbols = (x_var_symbols, )
R = LaurentPolynomialRing(K, x_var_symbols, nvar, order='lex')
s = Sym.schur()
K.inject_variables()
R.inject_variables()

Defining t
Defining x1, x2, x3, x4


In [23]:
def pad(p, n):
    ''' Return list of length n with padded 0's if necessary.
        
        Input:
            - ``p`` -- A Partition or list of integers.
    '''
    return p + [0 for i in xrange(n-len(p))]

def to_sym(poly, lengths=nvar):
    ''' Return symmetric polynomial ``poly`` in Schur basis, with number of parts <= ``lengths``. 
        
        Input:
            - ``poly`` -- an element of LaurentPolynomialRing.
        
        Warning:
            - Doesn't check input is symmetric
    '''
    return s.from_polynomial(poly, check=False).restrict_partition_lengths(lengths, exact=False)

def _inversion_pairs_from_position(mst, k, ij):
    ''' Return the number of inversions at the cell position `(i,j)` in the
        ``k``-th tableaux in ``mst``.
        
        Auxiliary function to compute inversion pairs. 
        Needed to rewrite this because there was an error in the Sage code.
    '''
    pk = k
    pi,pj = ij
    c = pj - pi
    value = mst[pk][pi][pj]
    same_diagonal  = [ t.cells_by_content(c) for t in mst[pk+1:] ]
    above_diagonal = [ t.cells_by_content(c+1) for t in mst[:pk] ]

    res = []
    for k in range(len(same_diagonal)):
        for i,j in same_diagonal[k]:
            if value > mst[pk+k+1][i][j]:
                res.append( ((pk,(pi,pj)), (pk+k+1,(i,j))) )
    for k in range(len(above_diagonal)):
        for i,j in above_diagonal[k]:
            if value > mst[k][i][j]:
                res.append( ((pk,(pi,pj)), (k,(i,j))) )
    return res 

def inversion_pairs(mst):
    ''' Return a list of inversion pairs in MultiSkewTableau ``mst``.
        
        Output:
            -- list of tuples that are the pairs. Each element in the tuple is the tuple (k, (i,j))
               which corresponds to the cell in row,col (i,j) in shape k.
    '''
    inv = []
    for k in range(len(mst)):
        for b in mst[k].cells():
            inv += _inversion_pairs_from_position(mst,k,b)
    return inv

def inversions(mst):
    return len(inversion_pairs(mst))

def parse_shapes(mu):
    ''' Return either list of partitions or list of skew partitions. '''
    try: # test if shapes are straight
        mu = [Partition(_) for _ in mu]
    except ValueError: # shapes must be skew
        mu = [SkewPartition(_) for _ in mu]
    except TypeError: # shapes are integers
        mu = [ Partition([_]) for _ in mu]
    except:
        raise InputError("mu needs to be a shape, a list of shapes, or a list of skew shapes")
    return mu

def LLT_inv(mu):
    ''' Return inversion LLT polynomial.
    
        Input:
            - ``mu`` -- A shape or list of shapes. If a shape, then computes LLT polynomial
              indexed by the rows of ``mu``. If a list of shapes, can be skew shapes or straight shapes.
              
        Output:
            - A symmetric polynomial written in the Schur basis, truncated to partitions with length <= nvar.
    '''
    mu = parse_shapes(mu)
    try:
        SSYT = [SemistandardTableaux(Partition(shape), max_entry=nvar) for shape in mu]
    except: # mu is list of skew shapes
        SSYT = [SemistandardSkewTableaux(list(shape), max_entry=nvar) for shape in mu]
    res = 0
    for ssyt in product(*SSYT):
        mst = MultiSkewTableau(list(ssyt))
        res += t^(inversions(mst)) * R.monomial(*pad(mst.weight(), nvar))                   
    return to_sym(res, lengths=nvar)

# Examples

In [9]:
# Input can be a single shape, which represents a tuple indexed by rows
mu = [3,1,1]
LLT_inv(mu)

t^3*s[3, 1, 1] + t^2*s[3, 2] + (t^2+t)*s[4, 1] + s[5]

In [27]:
# Input can be a list of straight shapes (assumed to be centered at 0 content line)
mu = [ [3,2], [1,1], [1] ]
LLT_inv(mu)

t^6*s[3, 2, 2, 1] + (t^6+t^5)*s[3, 3, 1, 1] + t^5*s[3, 3, 2] + (t^6+t^5)*s[4, 2, 1, 1] + t^5*s[4, 2, 2] + (t^5+2*t^4)*s[4, 3, 1] + t^3*s[4, 4] + t^4*s[5, 2, 1] + t^3*s[5, 3]

In [11]:
# Input can be a list of skew shapes
mu = [ [[2,1],[0]], [[2,2],[1]] ]
LLT_inv(mu)

t^4*s[2, 2, 1, 1] + t^4*s[2, 2, 2] + t^3*s[3, 1, 1, 1] + 2*t^3*s[3, 2, 1] + t^2*s[3, 3] + t^2*s[4, 1, 1] + t^2*s[4, 2]

### Built-in methods
For reference, here is how you compute LLT polynomials with the built-in methods. In my experience, this can take a long time for big partitions or a long list of shapes, but otherwise is faster than my code.

In [29]:
# If the input is a list of straight or skew shapes
mu = [ [3,2], [2], [1,1] ]
G = Sym.llt(len(mu))
s(G.cospin(mu))


t^5*s[3, 2, 2, 1, 1] + t^5*s[3, 3, 1, 1, 1] + 2*t^4*s[3, 3, 2, 1] + t^3*s[3, 3, 3] + t^4*s[4, 2, 1, 1, 1] + (t^4+t^3)*s[4, 2, 2, 1] + (t^4+2*t^3)*s[4, 3, 1, 1] + (2*t^3+t^2)*s[4, 3, 2] + (t^3+t^2)*s[4, 4, 1] + (t^3+t^2)*s[5, 2, 1, 1] + t^2*s[5, 2, 2] + (2*t^2+t)*s[5, 3, 1] + t*s[5, 4] + t*s[6, 2, 1] + s[6, 3]